In [ ]:
from tensorflow import keras

from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os


In [ ]:
path = '../input/devnagri-script-classification/Data'

In [ ]:
os.listdir('../input/devnagri-script-classification/Data')


In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255, horizontal_flip = False, validation_split = 0.13)

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)


In [ ]:
train_set = train_datagen.flow_from_directory(path + '/Train', target_size = (64, 64), batch_size = 32, class_mode = 'categorical', subset='training')


In [ ]:
validation_set = train_datagen.flow_from_directory(path + '/Train', target_size = (64, 64), batch_size = 32, class_mode = 'categorical', subset='validation')

In [ ]:
Labels = train_set.class_indices
print(Labels)
num_classes = len(Labels)

In [ ]:
Characters = 'क ख ग घ ङ च छ ज झ ञ ट ठ ड ढ ण त थ द ध न प फ ब भ म य र ल व श ष स ह क्ष त्र ज्ञ ० १ २ ३ ४ ५ ६ ७ ८ ९'

In [ ]:
Characters = Characters.split(' ')


In [ ]:
key_list = list(Labels.keys())

In [ ]:
import re
def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)',text) ]
ls = os.listdir(path+'/Train')
ls.sort(key = natural_keys)

In [ ]:
print(ls,end  ='')


In [ ]:
Cha_Uni = {}
for j,i in enumerate(ls):
    Cha_Uni[i] = Characters[j]

In [ ]:
model = Sequential()
model.add(Conv2D(64, (3, 3), padding='same', input_shape=(64, 64, 3), activation = 'relu'))
model.add(Conv2D(64, (3, 3), activation = 'relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(num_classes, activation = 'softmax'))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
history = model.fit(train_set, epochs=10, verbose=1, validation_data = validation_set, shuffle=True)

In [ ]:
import matplotlib.pyplot as plt


# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
train_path = '../input/devnagri-script-classification/Data/Train'
test_path = '../input/devnagri-script-classification/Data/Test'

In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("model.h5")
print("Saved model to disk")

In [ ]:
import tensorflow as tf
test_list = tf.io.gfile.listdir('../input/devnagri-script-classification/Data/Test')
imgs = []
for f in test_list:
    img =tf.keras.preprocessing.image.load_img('../input/devnagri-script-classification/Data/Test/'+ f,color_mode="rgb", target_size = (64,64))
    img = tf.keras.preprocessing.image.img_to_array(img)
    imgs.append(img)

In [ ]:
pred = model.predict(tf.convert_to_tensor(imgs))

In [ ]:
import numpy as np
pred = np.argmax(pred, axis=1)

In [ ]:
pred.shape

In [ ]:
class_indices = train_set.class_indices
len(class_indices)
class_indices = dict([(value,key) for key, value in class_indices.items()])

pred = list(map(class_indices.get, pred))

In [ ]:
import pandas as pd
submission = pd.DataFrame({'id': test_list,'category': pred})
submission

In [ ]:
submission

In [ ]:
submission.to_csv('submission.csv',index='False')